In [231]:
%reset -f

# AIML CA1

## Import General Dependencies

In [ ]:
# Mathematical Dependency
import numpy as np

# Data Manipulation Dependency
import pandas as pd

# Graphing Dependencies
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Dependencies
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline

# Preservation Dependency
import pickle

# Miscellaneous Dependencies
from typing import Callable, Dict, Union    # static typing
from warnings import filterwarnings         # warnings

In [ ]:
%matplotlib inline

In [ ]:
# Hide Warnings
filterwarnings(action='ignore')

## Part A > Classification

*   How is your prediction task defined? And what is the meaning of the
output variable?

```
    The task is to predict if a mushroom of the agaricus lepiota family is edible or poisonous,
    given its properties (i.e. cap-shape, odor, etc.)

    The output variable is class, and its possible values carry the respective meanings:
    'edible':       the mushroom is safe for consumption
    'poisonous':    do not consume the mushroom
```

### Import Data

Load data about edibility of gilled mushrooms of the agaricus lepiota family

*   How do you represent your data as features?

```
    The features are represented as columns in a pandas DataFrame
```

*   Did you bring in any additional sources of data?

```
    No, no external data sources were used
```

In [ ]:
from utils.extraction import extract_attributes

In [ ]:
def load_mushrooms() -> pd.DataFrame:
    # Extract raw content of ./data/agaricus-lepiota.names file
    metadata: str
    with open('./data/agaricus-lepiota.names') as f:
        metadata = f.read()

    # Extract attributes from metadata
    attrs = extract_attributes(metadata, r'7\. Attribute Information:.*\n((.|\n)*)8\. Missing')

    # Extract column names to be used for dataframe
    cols = attrs.keys()

    # Create the dataframe from ./data/agaricus-lepiota.data file,
    #   using column names derived from ./data/agaricus-lepiota.names file
    df = pd.read_csv(
        filepath_or_buffer='./data/agaricus-lepiota.data',
        sep=',',
        header=0,
        names=cols
    )

    # Expand attribute codes to their full definitions
    for col in cols:
        df[col].replace(to_replace=attrs[col] ,inplace=True)
    
    return df

In [ ]:
df = load_mushrooms()

In [ ]:
X_whole = df.drop(labels='class', axis=1)
y_whole = df['class']
X_build, X_final, y_build, y_final = train_test_split(X_whole, y_whole, test_size=0.2, random_state=1)
df_build = pd.concat(objs=(X_build, y_build), axis=1)

#### Inspect Data

In [ ]:
# Inspect top 10 rows of the dataset
df.head(n=10)

#### Summarize Data

In [ ]:
# Inspect overview of the dataset
df.info()

### Pre-Processing

*   Did you process the features in any way?

```
    Yes, the features underwent (feature) selection and (one-hot) encoding
```

#### Exploratory Data Analysis (EDA)

In [ ]:
# List to keep track of variables to be removed
drop_cols = []

Missing Values

In [ ]:
# Check for missing values
print(df.isna().sum(axis=0), end='\n')

# Note that stalk-root has missing attributes (denoted as 'missing')
# In fact, approx. 31% of the records have missing data for stalk-root
stalk_dist = df['stalk-root'].value_counts()
print((stalk_dist / stalk_dist.sum()).round(2))

# Course of action - drop column
drop_cols.append('stalk-root')

Redundant Features

In [ ]:
# Inspect unqiue counts of the individual features
print(df.describe().transpose().sort_values(by='unique', ascending=False))

# Note that veil-type has only one value,
#   hence it is a redundant feature
print('\n', 'Unique values for \'veil-type\': ', pd.unique(df['veil-type'].values), sep='')

# Course of action - drop column
drop_cols.append('veil-type')

Inspect the distribution of the target variable (class: edible/poisonous)

The data seems rather balanced, with an almost even distribution of edible and poisonous mushrooms.
This means that the inherent bias of the machine learning model is low.

For example, if there was much more data for edible mushrooms, the machine learning model would predict edible mushrooms much more precisely than poisonous ones.

In [ ]:
from utils.plotting import format_label

In [ ]:
def plot_1A():
    global df
    ax, fig = plt.subplots(figsize=(7, 5))
    ax = sns.countplot(data=df, x='class', palette='deep')
    ax.set_ylim(top=5000)
    ax.set_title(label='General Data Distribution')
    ax.set_ylabel(ylabel='Number of Records')
    ax.set_yticklabels(labels=format_label(ax.get_yticks() / 1000, lambda s: f'{round(s)}k'))
    ax.set_xlabel(xlabel='Type')
    total_count = df.shape[0]
    for p in ax.patches:
        x = p.get_x()
        y = p.get_height()
        ax.annotate(text=f'{y} ({y / total_count * 100:.1f}%)', xy=(x + 0.23, y + 70))

plot_1A()

Inspect correlation between the independent variables and the target variable (class)

The plot on the left depicts a chi2-based correlation between the distribution of the independent variables and the 'class' target variable.

This is how it works:
*   The more the data is evenly distributed between 'edible' and 'poisonous' for a certain attribute, the less it is correlated to edibility
*   For example, among 1,000 mushrooms with pink gills, if 500 are edible and 500 are poisonous, then pink gills is not a good indicator of whether a mushroom is poisonous
*   Any deviation from 50% distribution would hint some form of correlation

No columns/features were removed solely based on their correlations with the target ('class')

In [ ]:
def plot_1B():
    global df
    for i in df.drop(labels='class', axis=1).columns.values:
        fig, (corr_plot, freq_plot) = plt.subplots(ncols=2, figsize=(14, 6))
        ct = pd.crosstab(index=df['class'], columns=df[i])
        distr = df.groupby(i).count().iloc[:,0]
        proportion = ((ct.loc['edible']) / distr) - 0.5
        corr = pd.DataFrame(proportion.reset_index())
        sns.barplot(data=corr, x=i, y=0, ax=corr_plot, color='grey')
        sns.countplot(data=df.sort_values(by=i), x=i, hue='class', hue_order=['edible', 'poisonous'], ax=freq_plot, palette='turbo')
        fig.suptitle(t=f'{i.upper()}')
        corr_plot.set_title(label='Correlation (chi2-based)')
        corr_plot.set_ylim((-0.6, 0.6))
        corr_plot_x_lim = corr_plot.get_xlim()
        corr_plot.set_yticks(ticks=np.arange(-0.5, 0.6, 0.1))
        corr_plot.set_yticklabels(labels=np.round(corr_plot.get_yticks() + 0.5, 1))
        corr_plot.set_ylabel(ylabel='Correlation')
        corr_plot.set_xticklabels(labels=corr_plot.get_xticklabels(), rotation=30)
        corr_plot.annotate(text='Tends to be \'edible\'', xy=((corr_plot_x_lim[0] + corr_plot_x_lim[1]) / 2, 0.55), ha='center', color='green')
        corr_plot.annotate(text='Tends to be \'poisonous\'', xy=((corr_plot_x_lim[0] + corr_plot_x_lim[1]) / 2, -0.55), ha='center', color='orange')
        freq_plot.set_title(label=f'Frequency Distribution')
        freq_plot.set_xticklabels(labels=freq_plot.get_xticklabels(), rotation=30)

plot_1B()

#### Feature Engineering

There is no need to create any new features in this dataset

#### Feature Selection

There are 2 columns to be removed (stalk-root, veil-type):

1.  Stalk-root, because there are many missing values (31%)
2.  Veil-type, because it contains only 1 unique category

In [ ]:
# Remove columns
df.drop(labels=drop_cols, axis=1, inplace=True)
df_build.drop(labels=drop_cols, axis=1, inplace=True)

### Encoding the data

The data has only categorical text variables, therefore they have to be converted to numeric form.

Between label encoding and one-hot encoding, one-hot encoding makes more sense as the categorical variables are nominal rather than ordinal.

In [ ]:
# (One-Hot) Encode the dataset (categorical -> binary)
df_build = pd.get_dummies(data=df_build, drop_first=True)

In [ ]:
# Ensure all the possible feature values are accounted for
print(f'Shape of data subset:\t\t{df_build.shape}')
print(f'Shape of entire dataset:\t{pd.get_dummies(data=df, drop_first=True).shape}')

### Inspect correlation after encoding

The absence of odour seems to be very negatively correlated to a mushroom being poisonous.

Perhaps it would be a good indicator of a mushroom being edible.

In [ ]:
# Get correlation between top 10 factors and target variable (class)
df_build.corr()['class_poisonous'].drop(labels='class_poisonous').sort_values(key=lambda x: np.abs(x), ascending=False).head(n=10)

In [ ]:
# Chi2-based feature selection
from sklearn.feature_selection import chi2, SelectKBest

# Get top 10 factors that are correlated with the target variable (class)
best_features_chi2 = SelectKBest(score_func=chi2, k=10).fit(X=df_build.drop(labels='class_poisonous', axis=1), y=df_build['class_poisonous'])
best_features_mask = best_features_chi2.get_support()
best_features = df_build.drop(labels='class_poisonous', axis=1).columns.values[best_features_mask]
best_features_scores = best_features_chi2.scores_[best_features_mask]
good_predictors = pd.Series(data=best_features_scores, index=best_features)

good_predictors.sort_values(ascending=False)

### Data Partitioning

Split the build data into X and y

In [ ]:
# Split the dataset into training and test sets
X = df_build.drop(labels='class_poisonous', axis=1)
y = df_build['class_poisonous']

### Algorithm Selection & Hyper-Parameter Tuning

#### Determine best candidate algorithm using GridSearch

*   How did you select which learning algorithms to use?

```
    GridSearchCV was used, and the algorithm used was included within the parameter space.
    For each algorithm, 6 different combinations of key hyperparameters were tested.
    Out of all the combinations, the best performing algorithm was selected.
    In this case, the algorithm is logistic regression.
```

In [ ]:
# Candidate classification algorithms
from sklearn.naive_bayes import CategoricalNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Parent classes for custom transformers
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# Define a placeholder estimator class for use with GridSearchCV
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

In [ ]:
def grid_search_clf():
    pipe = Pipeline(steps=[
        ('clf', DummyEstimator())
    ])

    params = [
        {
            'clf': [KNeighborsClassifier()],
            'clf__n_neighbors': np.arange(3, 14, 2)
        },
        {
            'clf': [LogisticRegression()],
            'clf__solver': ['liblinear', 'newton-cg'],
            'clf__C': np.logspace(-3, 3, 3),
            'clf__multi_class': ['ovr']
        },
        {
            'clf': [CategoricalNB()],
            'clf__alpha': np.logspace(-3, 3, 6)
        },
        {
            'clf': [SVC()],
            'clf__kernel': ['rbf', 'linear'],
            'clf__C': np.logspace(-3, 4, 3)
        },
        {
            'clf': [DecisionTreeClassifier()],
            'clf__max_depth': [10, 20, 30],
            'clf__min_samples_leaf': [10, 30]
        }
    ]

    best_clf_algo = GridSearchCV(estimator=pipe, param_grid=params, cv=3)
    best_clf_algo.fit(X=X, y=y)
    return best_clf_algo

In [ ]:
# Save result
# pickle.dump(obj=grid_search_clf(), file=open('./models/best_clf_algo.p', 'wb'))

# Load result
best_clf_algo_loaded = pickle.load(file=open('./models/best_clf_algo.p', 'rb'))

# Inspect result
print(best_clf_algo_loaded.best_estimator_)
gs_clf = pd.DataFrame(best_clf_algo_loaded.cv_results_)
gs_clf.sort_values(by='rank_test_score')

#### Determine best hyperparameters for selected algorithm using GridSearch

Selected Algorithm: Logistic Regression

*   Did you try to tune the hyper parameters of the learning algorithm, and in that case how?

```
    Yes. GridSearchCV was used once again, but since the logistic regression algorithm
    had already been selected, more hyperparameters specific to logistic regression could be tested.
    
    Standardization was also compared against no scaling.
```

In [ ]:
class DummyScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return X

SVC performs as well as before, with an accuracy of roughly 1.0.

In [ ]:
def svc_tuning():
    global X, y
    pipe = Pipeline(steps=[
        ('clf', SVC())
    ])

    params = {
        'clf__kernel': ['linear'],
        'clf__C': np.logspace(-3, 3, 3),
        'clf__tol': [1e-5, 1e-4]
    }

    return GridSearchCV(estimator=pipe, param_grid=params, cv=3).fit(X=X, y=y)

In [ ]:
# Get result
svc_tuned = svc_tuning()

# Inspect result
print(svc_tuned.best_params_)
pd.DataFrame(svc_tuned.cv_results_).sort_values(by='rank_test_score')

In [ ]:
def grid_search_clf_params():
    global X, y
    pipe = Pipeline(steps=[
        ('scaler', DummyScaler()),
        ('clf', LogisticRegression())
    ])

    params = {
            'scaler': ['passthrough', StandardScaler()],
            'clf__solver': ['liblinear', 'saga'],
            'clf__tol': np.logspace(-5, 2, 3),
            'clf__C': np.logspace(-4, 4, 5),
            'clf__multi_class': ['ovr']
    }

    best_clf_params = GridSearchCV(estimator=pipe, param_grid=params, cv=5, n_jobs=-1)
    best_clf_params.fit(X=X, y=y)
    return best_clf_params

In [ ]:
# Save result
# pickle.dump(obj=grid_search_clf_params(), file=open('./models/best_clf_params.p', 'wb'))

# Load result
best_clf_params = pickle.load(file=open('./models/best_clf_params.p', 'rb'))

# Inspect result
print(best_clf_params.best_params_)
gs_clf_params = pd.DataFrame(best_clf_params.cv_results_)
gs_clf_params.sort_values(by='rank_test_score')

No scaler is needed.

### Combining Everything

<a id="classification-finale" /></a>

### Building Pipeline
<br>
Build a machine learning pipeline, using

*   a custom feature-selection transformer,
*   a one-hot encoder,
*   the most consistent algorithm,
*   the best performing hyperparameters

In [ ]:
def drop_redundant_cols_1(df: pd.DataFrame):
    return df.drop(labels=drop_cols, axis=1, errors='ignore')

def ensure_target_absence_1(df: pd.DataFrame):
    return df.drop(labels='class', axis=1)

class FeatureSelector1(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.has_target_variable: bool = False

    def fit(self, X, y=None):
        if 'class' in X.columns.values:
            self.has_target_variable = True
        return self
    
    def transform(self, X):
        X_copy = drop_redundant_cols_1(X)
        if self.has_target_variable:
            X_copy = ensure_target_absence_1(X_copy)
        return X_copy

In [ ]:
# Import encoder
from sklearn.preprocessing import OneHotEncoder

# Specify all possible column values for dataset
def get_column_values_1(df: pd.DataFrame):
    categories = []
    for i in df.drop(labels=['class', *drop_cols], axis=1, errors='ignore').columns.values:
        categories.append(pd.unique(df[i]))
    return categories

In [ ]:
model = Pipeline(steps=[
    ('feature_selector', FeatureSelector1()),
    ('encoder', OneHotEncoder(drop='first', categories=get_column_values_1(df))),
    ('classifier', LogisticRegression(C=100.0, multi_class='ovr', solver='liblinear', tol=1e-05))
])

### Model Training

Fit the data to the pipeline

In [ ]:
model.fit(X=X_build, y=y_build)

In [ ]:
# Save the model
# pickle.dump(obj=model, file=open('./models/final_classifier.p', 'wb'))

In [ ]:
# Load the model
final_classifier = pickle.load(file=open('./models/final_classifier.p', 'rb'))

### Model Scoring

Use the model to generate predictions

In [ ]:
y_pred = final_classifier.predict(X=X_final)
y_pred

### Model Evaluation

Evaluate the performance of the final model based on standard classification metrics

*   How do you evaluate the quality of your system?

```
    The machine learning pipeline was evaluated against:
    1.  the train set,
    2.  the test set,
    3.  the entire dataset

    in terms of:
    1.  accuracy,
    2.  precision,
    3.  recall,
    4.  f1-score
```

In [ ]:
# Import model evaluation dependencies
from sklearn.metrics import classification_report, confusion_matrix

#### Evaluate against build set

In [ ]:
y_build_pred = final_classifier.predict(X=X_build)

# Classification summary
print(classification_report(y_true=y_build, y_pred=y_build_pred, target_names=['edible', 'poisonous']))

# Confusion matrix
print('\n', pd.DataFrame(data=confusion_matrix(y_true=y_build, y_pred=y_build_pred, labels=['edible', 'poisonous']), index=['Actual Edible', 'Actual Poisonous'], columns=['Predicted Edible', 'Predicted Poisonous']), '\n\n', sep='')

# Print target distribution in y_build
print(y_build.groupby(y_build).count())

#### Evaluate against final set

*   How do you evaluate the quality of your system?

```
    To check for overfitting, the accuracy score for the build and final sets were compared.
    They were approximately within 1% of each other (~0.99 and ~0.99)
```

*   How well does your system compare to a stupid baseline?

```
    My machine learning pipeline predicts correctly (~0.99) almost
    twice as frequently as compared to a stupid baseline (~0.50)
```

*   Can you say anything about the errors that the system makes? For a classification task, you may consider a confusion matrix.

```
    My machine learning pipeline is quite accurate, so there are few to no errors.
    Using the confusion matrix as seen below, all the samples (mushrooms) in the test set
    were correctly categorised as 'edible' or 'poisonous' respectively.

    Recall is also 1.00. That means that all the poisonous mushrooms were successfully identified.
```

In [ ]:
# Classification summary
print(classification_report(y_true=y_final, y_pred=y_pred, target_names=['edible', 'poisonous']))

# Confusion matrix
print('\n', pd.DataFrame(data=confusion_matrix(y_true=y_final, y_pred=y_pred, labels=['edible', 'poisonous']), index=['Actual Edible', 'Actual Poisonous'], columns=['Predicted Edible', 'Predicted Poisonous']), '\n\n', sep='')

# Print target distribution in y_final
print(y_final.groupby(y_final).count())

In [ ]:
# Dummy baseline for classification
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='uniform')
dummy.fit(X_build, y_build)
dummy.score(X_final, y_final)

#### Evaluate against entire dataset

In [ ]:
y_whole_pred = final_classifier.predict(X=X_whole)

# Classification summary
print(classification_report(y_true=y_whole, y_pred=y_whole_pred, target_names=['edible', 'poisonous']))

# Confusion matrix
print('\n', pd.DataFrame(data=confusion_matrix(y_true=y_whole, y_pred=y_whole_pred, labels=['edible', 'poisonous']), index=['Actual Edible', 'Actual Poisonous'], columns=['Predicted Edible', 'Predicted Poisonous']), '\n\n', sep='')

# Print target distribution in y_whole
print(y_whole.groupby(y_whole).count())

*   Is it possible to say something about which features the model considers
important? (Whether this is possible depends on the type of classifier
you are using)

```
    The model does produce numerical coefficients, but these are not directly indicative of the
    feature importances, as the features have been encoded.
```

In [ ]:
def get_feature_coefs_1():
    global df, final_classifier
    coefs = final_classifier['classifier'].coef_.reshape((-1,))
    final_features = pd.get_dummies(FeatureSelector1().fit_transform(df), drop_first=True).columns.values
    feature_coefs = pd.Series(data=coefs, index=final_features)
    return feature_coefs

get_feature_coefs_1().sort_values(ascending=False, key=lambda x: np.abs(x))

## Part B > Regression

*   How is your prediction task defined? And what is the meaning of the output variable?

```
    The task is to predict the selling price of a house,
    given its properties (i.e. number of bedrooms, floor area, etc.)

    The output variable is measured in USD, and is the predicted price of a house in King County
```

### Import Data

Load data about King County house sales

*   How do you represent your data as features?

```
    The features are represented as columns in a pandas DataFrame
```

*   Did you bring in any additional sources of data?

```
    No, no external sources of data were used
```

In [ ]:
# Read the data from a csv file
df2 = pd.read_csv('./data/kc_house_data.csv')

In [ ]:
X2_whole = df2.drop(labels='price', axis=1)
y2_whole = df2['price']
X2_build, X2_final, y2_build, y2_final = train_test_split(X2_whole, y2_whole, test_size=0.2, random_state=4)
df2_build = pd.concat(objs=(X2_build, y2_build), axis=1)

#### Inspect Data

Preview a sample of the dataset

In [ ]:
# Inspect the top 10 rows of the dataset
df2.head(n=10)

#### Summarize Data

Get a sense of the features involved

In [ ]:
# Inspect overview of the dataset
df2.info()

In [ ]:
# Inspect statistics of the dataset
df2.describe().transpose().round(2)

### Pre-Processing

*   Did you process the features in any way?

```
    Yes, the features underwent (feature) engineering and selection,
    logarithmic transformation, and standardization.

    In addition, the target also underwent square root transformation.
```

#### Exploratory Data Analysis (EDA)

In [ ]:
# List to keep track of variables to be removed
drop_cols_2 = []

# List to keep track of positively skewed variables
positively_skewed = []

In [ ]:
# Check for missing values
df2.isna().sum(axis=0)

# There doesn't seem to be any missing values

Visualize correlation amongst the original features using a heatmap

`sqft_living`, `sqft_above`, `sqft_basement` and `sqft_lot` seem interrelated, especially `sqft_living` and `sqft_above` (suspiciously high correlation). As defined [here](https://rstudio-pubs-static.s3.amazonaws.com/155304_cc51f448116744069664b35e7762999f.html), `sqft_living` is the total interior floor area and it encapsulates `sqft_above`, `sqft_basement` and `sqft_lot`. To reduce collinearity, not all these features should be used to reduce overfitting.

The id feature is not correlated to any other feature. If it is just an ordinary sequential id tag, it might be useless for our purposes as there isn't anything to generalize from id.

In [ ]:
def plot_2A():
    global df2
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(data=df2.corr(), cmap='RdBu', vmin=-1, vmax=1, ax=ax)
    ax.set_title(label='Correlation Matrix')
plot_2A()

In [ ]:
# Interrelated features may lead to overfitting

# Course of action - drop columns
drop_cols_2.extend(['sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15'])

Inspect distribution of the individual variables

It can be seen that a few variables are positively skewed, such as `sqft_living` and `price`. Perhaps some (logarithmic/root) transformation could be applied to the data to make it conform more to a Normal distribution, which would improve the performance of most machine learning models.

ID seems quite randomly distributed.

In [ ]:
def plot_2B():
    global df2
    for i in df2.columns.values:
        if df2[i].dtype.kind in 'biufc':
            with sns.axes_style(style='darkgrid'):
                fig, (hst, bxp) = plt.subplots(ncols=2, figsize=(12, 5))
                fig.suptitle(i.upper())
                sns.histplot(data=df2, x=i, ax=hst, palette='deep')
                sns.boxplot(data=df2, x=i, ax=bxp, palette='deep')
plot_2B()

In [ ]:
# Many of the features seem to be positively skewed

# Course of action - log/sqrt transformation
positively_skewed.extend(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'long', 'sqft_living15', 'sqft_lot15'])

Inspect distributions of the transformed features, including `price`

Logarithmic transformation seems to have the greatest impact in conforming the data to a Normal distribution. Nevertheless, we would have to test out both (log/root) transformations in a grid search to see which actually performs better.

In [ ]:
def plot_2C():
    global df2, positively_skewed
    for i in positively_skewed:
        fig, (non, log, sqrt) = plt.subplots(ncols=3, figsize=(8, 3))
        non.set_title(label='No Transformation')
        log.set_title(label='After Log Transformation')
        sqrt.set_title(label='After Sqrt Transformation')
        sns.histplot(x=df2[i], ax=non)
        sns.histplot(x=np.log1p(df2[i]), ax=log)
        sns.histplot(x=np.sqrt(df2[i]), ax=sqrt)
        plt.subplots_adjust(wspace=0.4)
plot_2C()

In [ ]:
df2.corr()['price'].sort_values(key=lambda x: np.abs(x), ascending=False)

Inspect id feature

Id seems to be redundant in this case as it is merely a serialization tag.

In [ ]:
# Check id data type
print('ID data type:\t\t', df2['id'].dtype)

# Compare the number of ids to the total number of records 
print('Number of unique IDs:\t', pd.unique(df2['id']).size)
print('Total no. of records:\t', df2.shape[0], '\n')

# Check correlation between id and the rest of the variables
print(df2.corr()['id'].sort_values(key=lambda x: np.abs(x), ascending=False))


# id seems redundant

# Course of action - drop column
drop_cols_2.append('id')

Inspect zipcode feature

Zipcode may provide some information as to where the house is located or which neighbourhood it resides in. As some patterns could be derived from these data, it should not be removed.

In [ ]:
# Check zipcode data type
print('zipcode data type:\t\t', df2['zipcode'].dtype)

# Compare the number of zipcodes to the total number of records 
print('Number of unique zipcodes:\t', pd.unique(df2['zipcode']).size)
print('Total no. of records:\t\t', df2.shape[0], '\n')

# Check correlation between zipcode and the rest of the variables
print(df2.corr()['zipcode'].sort_values(key=lambda x: np.abs(x), ascending=False))


# zipcode does not seem redundant

# Course of action - no action

Inspect correlations between transformed features and untransformed target (`price`)

The shapes of the correlations change when logarithmic or square root transformations are applied on the feature variables. Specifically, logarithmically transformed `sqft_living` seems to be positively exponentially related to untransformed `price`.

No transformation should not be performed on `long` (longitude) as it yields no result.

To counter the effects of the X transformations, y (`price`) should be transformed too. Nonetheless, we would have to test them with grid search later on to see which indeed performs the best.

In [ ]:
def plot_2D():
    global df2, positively_skewed
    for i in positively_skewed:
        with sns.axes_style(style='whitegrid'):
            fig, (bef, log, sqrt) = plt.subplots(ncols=3, figsize=(10, 5))
            bef.set_title(label='No Transformation')
            log.set_title(label='Log Transformation')
            sqrt.set_title(label='Sqrt Transformation')
            sns.scatterplot(data=df2, x=i, y='price', ax=bef)
            sns.scatterplot(x=np.log1p(df2[i]), y=df2['price'], ax=log)
            sns.scatterplot(x=np.sqrt(df2[i]), y=df2['price'], ax=sqrt)
plot_2D()

#### Feature Engineering

There seems to be useful extractable data in the `date` feature, such as the `year`, `month` and `day` on which the house was sold

In [ ]:
df2['date']

In [ ]:
# Extract year, month and day from the date feature
df2_date = pd.to_datetime(df2['date'], yearfirst=True)
df2['year'] = pd.DatetimeIndex(data=df2_date).year
df2['month'] = pd.DatetimeIndex(data=df2_date).month
df2['day'] = pd.DatetimeIndex(data=df2_date).day

df2_build_date = pd.to_datetime(df2_build['date'], yearfirst=True)
df2_build['year'] = pd.DatetimeIndex(data=df2_build_date).year
df2_build['month'] = pd.DatetimeIndex(data=df2_build_date).month
df2_build['day'] = pd.DatetimeIndex(data=df2_build_date).day

# Date variable seems redundant now

# Course of action - drop column
drop_cols_2.append('date')

#### Feature Selection

Drop redundant columns

In [ ]:
# Review columns to be dropped
drop_cols_2

There are 6 columns to be removed (`id`, `date`, `sqft_lot`, `sqft_above`, `sqft_basement`, `sqft_lot15`):

1.  id, because it is redundant
2.  date, because its necessary components have been extracted
3.  sqft_lot, sqft_above and sqft_basement, because they are related to sqft_living
4.  sqft_lot15, because it is related to sqft_living15

In [ ]:
# Remove columns
df2.drop(labels=drop_cols_2, axis=1, inplace=True)
df2_build.drop(labels=drop_cols_2, axis=1, inplace=True)

### Data Partitioning

Split the build data into X2 and y2

In [ ]:
# Split the dataset into training and test sets
X2 = df2_build.drop(labels='price', axis=1)
y2 = df2_build['price']

### Algorithm Selection & Hyper-Parameter Tuning

Determine best regression algorithm using GridSearch

*   How did you select which learning algorithms to use?

```
    GridSearchCV was used, and the algorithm used was one of the parameters.
    For each algorithm, a maximum of 6 different combinations of key hyperparameters were tested.
    Out of all the combinations, the best performing algorithm was selected.
    
    In this case, the algorithm is gradient boosting regressor,
    which is an ensemble learning algorithm.
```

In [ ]:
# Candidate regression algorithms
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
def grid_search_reg():
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('reg', DummyEstimator())
    ])

    params = [
        {
            'reg': [LinearRegression()],
            'reg__normalize': [True, False],
            'reg__fit_intercept': [True, False]
        },
        {
            'reg': [Lasso(), Ridge()],
            'reg__alpha': np.logspace(-5, 3, 6)
        },
        {
            'reg': [DecisionTreeRegressor(), GradientBoostingRegressor()],
            'reg__max_depth': np.arange(5, 11)
        },
        {
            'reg': [KNeighborsRegressor()],
            'reg__n_neighbors': np.arange(5, 11)
        },
    ]

    best_reg_algo = GridSearchCV(estimator=pipe, param_grid=params, cv=3, n_jobs=-1)
    best_reg_algo.fit(X=X2, y=y2)
    return best_reg_algo

In [ ]:
# Save result
# pickle.dump(obj=grid_search_reg(), file=open('./models/best_reg_algo.p', 'wb'))

# Inspect result
best_reg_algo_loaded = pickle.load(file=open('./models/best_reg_algo.p', 'rb'))

print(best_reg_algo_loaded.best_estimator_)
gs_reg = pd.DataFrame(best_reg_algo_loaded.cv_results_)
gs_reg.sort_values(by='rank_test_score')

#### Determine best hyperparameters for selected algorithm using GridSearch

Selected Algorithm: Gradient Boosting Regressor

*   Did you try to tune the hyper parameters of the learning algorithm, and
in that case how?

```
    Yes. GridSearchCV was used once again, but since the gradient boosting regressor algorithm
    had already been selected, more hyperparameters specific to gradient boosting regressor could
    be tested.

    This was performed in 2 stages: a broad search and a finer tuning phase
    
    In this case, maximum depth, minimum samples for leaf nodes and for splits were tested.
```

In [ ]:
def grid_search_reg_params():
    global X2, y2
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('reg', GradientBoostingRegressor())
    ])

    params = {
        'reg__max_depth': np.arange(2, 5),
        'reg__min_samples_leaf': np.arange(9, 100, 30),
        'reg__min_samples_split': np.arange(9, 100, 30)
    }

    best_reg_params = GridSearchCV(estimator=pipe, param_grid=params, cv=3, n_jobs=-1)
    best_reg_params.fit(X=X2, y=y2)
    return best_reg_params

In [ ]:
# Save result
# pickle.dump(obj=grid_search_reg_params(), file=open('./models/best_reg_params.p', 'wb'))

# Inspect result
best_reg_params = pickle.load(file=open('./models/best_reg_params.p', 'rb'))

print(best_reg_params.best_params_)
gs_reg_params = pd.DataFrame(best_reg_params.cv_results_)
gs_reg_params.sort_values(by='rank_test_score')

After the broad search was conducted, it seems that

`{'reg__max_depth': 4, 'reg__min_samples_leaf': 9, 'reg__min_samples_split': 39}`

performs the best. The hyper-parameter space can be narrowed down to values closer to 9 and 39.

In [ ]:
def grid_search_reg_params_2():
    global X2, y2
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('reg', GradientBoostingRegressor())
    ])

    params = {
        'reg__max_depth': [4],
        'reg__min_samples_leaf': np.arange(7, 10),
        'reg__min_samples_split': [30, 40, 50]
    }

    best_reg_params = GridSearchCV(estimator=pipe, param_grid=params, cv=5, n_jobs=-1)
    best_reg_params.fit(X=X2, y=y2)
    return best_reg_params

In [ ]:
# Save result
# pickle.dump(obj=grid_search_reg_params_2(), file=open('./models/best_reg_params_2.p', 'wb'))

# Inspect result
best_reg_params_2 = pickle.load(file=open('./models/best_reg_params_2.p', 'rb'))

print(best_reg_params_2.best_params_)
gs_reg_params = pd.DataFrame(best_reg_params_2.cv_results_)
gs_reg_params.sort_values(by='rank_test_score')

Further tuning - Scaler, Normalizing X Transformer

Custom X transformers were designed, to conform the positively skewed columns to more Normal distributions

In [ ]:
class DummyTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return X

In [ ]:
class LogTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        all_cols = X.columns.values
        for i in positively_skewed:
            if i != 'long' and i not in drop_cols_2:
                X_copy[i] = np.log1p(X[i])
        return X_copy

class SqrtTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        all_cols = X.columns.values
        for i in positively_skewed:
            if i != 'long' and i not in drop_cols_2:
                X_copy[i] = np.sqrt(X[i])
        return X_copy

In [ ]:
def best_transformer():
    pipe = Pipeline(steps=[
        ('trans', DummyTransformer()),
        ('scaler', DummyScaler()),
        ('reg', GradientBoostingRegressor(
            max_depth=4,
            min_samples_leaf=7,
            min_samples_split=30
        ))
    ])

    params = {
        'trans': ['passthrough', LogTransformer(), SqrtTransformer()],
        'scaler': ['passthrough', StandardScaler(), RobustScaler()]
    }

    best_trans_params = GridSearchCV(estimator=pipe, param_grid=params, cv=5, n_jobs=-1)
    best_trans_params.fit(X=X2, y=y2)
    return best_trans_params

In [ ]:
# Save result
# pickle.dump(obj=best_transformer(), file=open('./models/best_reg_trans.p', 'wb'))

# Inspect result
best_reg_trans = pickle.load(file=open('./models/best_reg_trans.p', 'rb'))

print(best_reg_trans.best_params_)
gs_reg_trans = pd.DataFrame(best_reg_trans.cv_results_)
gs_reg_trans.sort_values(by='rank_test_score')

Further tuning - Normalizing y Transformer

To counter the effects of X transformation, the target variable (y) could be transformed too

In [ ]:
from sklearn.compose import TransformedTargetRegressor

In [ ]:
def further_tune_reg(cv: int = 4):
    pipe = Pipeline(steps=[
        ('trans', LogTransformer()),
        ('scaler', RobustScaler()),
        ('reg', GradientBoostingRegressor(max_depth=4, min_samples_leaf=7, min_samples_split=30))
    ])

    sqrt_y = TransformedTargetRegressor(regressor=pipe, func=np.sqrt, inverse_func=np.square)

    log_y = TransformedTargetRegressor(regressor=pipe, func=np.log1p, inverse_func=np.expm1)

    scores = []
    for m in (pipe, sqrt_y, log_y):
        scores.append(cross_val_score(estimator=m, X=X2, y=y2, cv=cv))
    result = pd.DataFrame(data=scores, columns=[f'Test {i + 1}' for i in range(cv)], index=['no y transformation', 'sqrt y transformation', 'log y transformation'])
    result['Mean Score'] = result.mean(axis=1)
    result['Std Score'] = result.std(axis=1)
    return result

In [ ]:
# Save result
# pickle.dump(obj=further_tune_reg(), file=open('./models/best_reg_y_trans.p', 'wb'))

# Inspect result
best_reg_y_trans = pickle.load(file=open('./models/best_reg_y_trans.p', 'rb'))

print(best_reg_y_trans.sort_values(by='Mean Score', ascending=False))

### Combining Everything

<a id="regression-finale" /></a>

### Building the pipeline
<br>
Build the machine learning pipeline, using

*   a custom feature-engineering transformer,
*   a custom feature-selection transformer,
*   a custom logarithmic X transformer,
*   a robust scaler,
*   the most consistent algorithm (gradient boosting regressor),
*   the best performing hyperparameters

To further improve performance and reduce overfitting,<br>
the target variable will be transformed too (sqrt)

In [ ]:
def extract_date_parts(df: pd.DataFrame, col: str, **kwargs):
    df_datetime = pd.DatetimeIndex(df[col], **kwargs)
    return df_datetime.year, df_datetime.month, df_datetime.day

class FeatureEngineer2(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_year, X_month, X_day = extract_date_parts(df=X, col='date', yearfirst=True)
        X_copy['year'] = X_year
        X_copy['month'] = X_month
        X_copy['day'] = X_day
        return X_copy

In [ ]:
def drop_redundant_cols_2(df: pd.DataFrame):
    return df.drop(labels=drop_cols_2, axis=1, errors='ignore')

def ensure_target_absence_2(df: pd.DataFrame):
    return df.drop(labels='price', axis=1)

class FeatureSelector2(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.has_target_variable: bool = False

    def fit(self, X, y=None):
        if 'price' in X.columns.values:
            self.has_target_variable = True
        return self

    def transform(self, X):
        X_copy = drop_redundant_cols_2(X)
        if self.has_target_variable:
            X_copy = ensure_target_absence_2(X_copy)
        return X_copy

In [ ]:
# Build pipeline
pipe2 = Pipeline(steps=[
    ('feature_engineer', FeatureEngineer2()),
    ('feature_selector', FeatureSelector2()),
    ('sqrt_transformer', LogTransformer()),
    ('standard_scaler', RobustScaler()),
    ('regressor', GradientBoostingRegressor(max_depth=4, min_samples_leaf=7, min_samples_split=30))
])

In [ ]:
# Wrap pipeline in a target transformer
model2 = TransformedTargetRegressor(regressor=pipe2, func=np.sqrt, inverse_func=np.square, check_inverse=False)

### Model Training

Fit the data to the model

In [ ]:
model2.fit(X=X2_build, y=y2_build)

In [ ]:
# Save the model
# pickle.dump(obj=model2, file=open('./models/final_regressor.p', 'wb'))

In [ ]:
# Load the model
final_regressor = pickle.load(file=open('./models/final_regressor.p', 'rb'))

### Model Scoring

Use the model to generate predictions

In [ ]:
y_pred_2 = final_regressor.predict(X2_final)
y_pred_2

### Model Evaluation

Evaluate the performance of the final model based on standard regression metrics

*   How do you evaluate the quality of your system?

```
    The machine learning pipeline was evaluated against:
    1.  the train set,
    2.  the test set,
    3.  the entire dataset

    in terms of:
    1.  mean squared error,
    2.  mean absolute error,
    3.  r squared
```

In [ ]:
# Import regression metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
def regression_report(y_true, y_pred, type: str):
    print(
f'''Regression Report ({type})
================================
MSE:\t\t{np.round(mean_squared_error(y_true=y_true, y_pred=y_pred), 2)}
MAE:\t\t{np.round(mean_absolute_error(y_true=y_true, y_pred=y_pred), 2)}
R2:\t\t{np.round(r2_score(y_true=y_true, y_pred=y_pred), 4)}
''')

#### Evaluate against build set

In [ ]:
regression_report(y2_build, final_regressor.predict(X2_build), type='train')

#### Evaluate against final set

*   How do you evaluate the quality of your system?

```
    To check for overfitting, the r2 score for the build and final sets were compared.
    They were approximately within 5% of each other (~0.92 and ~0.88),
    which indicates that the model is not overfitted.
```

*   How well does your system compare to a stupid baseline?

```
    My machine learning pipeline predicts more accurately than a stupid baseline by a huge margin.

    The r2 score (which describes how well the model fits the data) of my machine
    learning pipeline is ~0.88 while that of a stupid baseline is negative
    (absolutely bad fit with respect to the data).
```

*   Can you say anything about the errors that the system makes?

```
    Concerning a regression problem, it is impossible to predict an exact value with machine learning.
    Therefore, the metrics by which a regression machine learning model is evaluated take into account
    the margin of error which the model makes. For example, r2 shows the proportion of variance
    (between a predicted value and the actual value) attributed to variance in the feature values.
    Hence, an r2 score close to 1 suggests success for a machine learning model.

    In this case 0.88 for a test score is considerably good.
```

In [ ]:
regression_report(y2_final, y_pred_2, type='test')

In [ ]:
# Dummy baseline for regression
from sklearn.dummy import DummyRegressor
dummy2 = DummyRegressor(strategy='median')
dummy2.fit(X2_build, y2_build)
dummy2.score(X2_final, y2_final)

#### Evaluate against entire dataset

In [ ]:
regression_report(y2_whole, final_regressor.predict(X2_whole), type='entire')

#### Evaluate against entire dataset (visualization)

The model seems to perform quite consistently within the 0.8 - 1.0 band, hovering around 0.9. For sample sizes of at least 500, the variation decreases further to produce a relatively stable horizontal line. 

In [ ]:
def plot_2E():
    df2_new = pd.read_csv('./data/kc_house_data.csv')
    fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(16, 10))

    for r in range(3):
        for c in range(3):
            df_tmp = df2_new.sample(frac=1)
            scores = []
            buffers = np.arange(100, 1000, 50)
            for buf in buffers:
                scores.append(final_regressor.score(df_tmp.drop('price', axis=1).iloc[:buf,:], df_tmp['price'].iloc[:buf]))
            sns.lineplot(x=buffers, y=scores, color='black', ax=ax[r,c])
            ax[r,c].set(
                title=f'Test {r * 3 + c + 1}',
                ylim=(0.4, 1.05),
                yticks=np.arange(0.5, 1.05, 0.1),
                ylabel='r2',
                xticks=np.arange(0, 1001, 100),
                xlabel='Buffer Size'
            )
            sns.lineplot(x=[0, 1000], y=[1.0] * 2, color='green', ax=ax[r,c])
            sns.lineplot(x=[0, 1000], y=[0.9] * 2, color='orange', ax=ax[r,c])
            sns.lineplot(x=[0, 1000], y=[0.8] * 2, color='red', ax=ax[r,c])
            sns.lineplot(x=[500] * 2, y=[0.4, 1.], color='grey', ax=ax[r,c])
    plt.subplots_adjust(hspace=0.45)

plot_2E()

*   Is it possible to say something about which features the model considers important?

```
    The more important features have greater values as seen below.
    It seems grade, floor area and geographical coordinates are the most important features.

    These roughly match our intuition, as floor area, quality of construction and design and
    location are crucial factors when deciding which house to purchase.
```

In [ ]:
def get_feature_importances_2():
    global X2_build, final_regressor
    impts = final_regressor.regressor_['regressor'].feature_importances_
    final_features = FeatureSelector2().fit_transform(FeatureEngineer2().fit_transform(X2_build)).columns.values
    feature_impts = pd.Series(data=impts, index=final_features)
    return feature_impts

get_feature_importances_2().sort_values(ascending=False)

## Conclusions

Every machine learning problem is different, and there is no one best algorithm to solve all.
It is the data scientist's responsibility to experiment and select the most appropriate algorithm that is able to generalize patterns in the available data, and make accurate and precise predictions on new, unseen data. Nonetheless, that is just the beginning as there are hyperparameters to tune, transformations to be performed and overfitting to check for.

In Part A, the [Logistic Regression algorithm](#classification-finale) was chosen, whereas in Part B, the [Gradient Boosting Regressor Ensemble algorithm](#regression-finale) was chosen instead. One-Hot Encoding was applied for Part A, while logarithmic and square root transformations were performed in Part B.